In [66]:
import requests
import pandas as pd
import uuid

In [93]:


params = {
    'appname': 'amcross'  
    ,'profile': 'full'
    ,'preset': 'latest'
    ,'limit': 100
    ,'query[fields][]':'format.name'
    ,'query[value]':'Situation Report'
    #if I remove this, seems like it returns everything
    #,'fields[include][]':['source',"format",'title',"body",'disaster.glide','primary_country']
}


api_endpoint = 'https://api.reliefweb.int/v1/reports?appname=amcross'

# Make the API request
response = requests.get(api_endpoint, params=params)

# Check the status of the response
if response.status_code == 200:
    # Parse and use the response data (in JSON format)
    data = response.json()
    situation_reports = data['data']

else:
    print(f"Error: {response.status_code} - {response.text}")

In [94]:
json_data = response.json()['data']
len(json_data)

100

In [100]:
def extract_themes(j):
    #must pass in after levelling json up to j['fields']
    themes = j.get('theme')
    if themes is None:
        return None
        
    ts =[]
    for theme in themes:
        ts.append(theme['name'].lower())
    return '; '.join(ts)

In [101]:
df_reliefweb_situation_report = pd.DataFrame(columns = ['record_type','source_url','glide_id','source_level_country','source_title','source_desc','source_original_text','reference_url','text','authoring_org','reported_date'])

for j in json_data:

    #parse 
    reference_url = j['href']
        
    j = j['fields']
    
    glide_id = None
    disaster = j.get('disaster')
    if disaster:
        glide_id = disaster[0]['glide']

    
    rec_id = j['id']
    title = j['title']
    original_text = j['body']
    link_to_doc = j['url_alias']
    file_url = j['file'][0]['url']
    primary_country_iso3 = j['primary_country']['iso3']
    primary_country = j['primary_country']['shortname']
    author_org = j['source'][0]['shortname']
    report_date = j['date']['original']
    themes = extract_themes(j)
   
    original_text_list = original_text.split("\n\n")
    for o in original_text_list:
        row = ['situation report',reference_url,glide_id,primary_country,title,themes,o,file_url,o,author_org,report_date]
        df_reliefweb_situation_report.loc[len(df_reliefweb_situation_report)] = row

In [102]:
df_reliefweb_situation_report

,record_type,source_url,glide_id,source_level_country,source_title,source_desc,source_original_text,reference_url,text,authoring_org,reported_date
0,situation report,https://api.reliefweb.int/v1/reports/4017691,None,World,Multi-country outbreak of mpox (monkeypox) - E...,health,**Highlights**,https://reliefweb.int/attachments/2e5a83c9-d6f...,**Highlights**,WHO,2023-11-25T00:00:00+00:00
1,situation report,https://api.reliefweb.int/v1/reports/4017691,None,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- The mpox surveillance reporting frequency ha...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- The mpox surveillance reporting frequency ha...,WHO,2023-11-25T00:00:00+00:00
2,situation report,https://api.reliefweb.int/v1/reports/4017691,None,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- A total of 668 new laboratory-confirmed case...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- A total of 668 new laboratory-confirmed case...,WHO,2023-11-25T00:00:00+00:00
3,situation report,https://api.reliefweb.int/v1/reports/4017691,None,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- Based on the data reported through global su...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- Based on the data reported through global su...,WHO,2023-11-25T00:00:00+00:00
4,situation report,https://api.reliefweb.int/v1/reports/4017691,None,World,Multi-country outbreak of mpox (monkeypox) - E...,health,- WHO has published a Disease Outbreak News ou...,https://reliefweb.int/attachments/2e5a83c9-d6f...,- WHO has published a Disease Outbreak News ou...,WHO,2023-11-25T00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...
1112,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,- Following the onset of the conflict in Sudan...,https://reliefweb.int/attachments/a94aa91b-f5d...,- Following the onset of the conflict in Sudan...,OCHA,2023-11-22T00:00:00+00:00
1113,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,- Of the 22 points of entry (PoEs) that are mo...,https://reliefweb.int/attachments/a94aa91b-f5d...,- Of the 22 points of entry (PoEs) that are mo...,OCHA,2023-11-22T00:00:00+00:00
1114,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,"- Currently, the onward transportation assista...",https://reliefweb.int/attachments/a94aa91b-f5d...,"- Currently, the onward transportation assista...",OCHA,2023-11-22T00:00:00+00:00
1115,situation report,https://api.reliefweb.int/v1/reports/4016821,EP-2023-000181-SDN,South Sudan,South Sudan: Response to the Sudan Crisis Situ...,contributions; coordination; food and nutritio...,- The number of Sudanese refugees and asylum s...,https://reliefweb.int/attachments/a94aa91b-f5d...,- The number of Sudanese refugees and asylum s...,OCHA,2023-11-22T00:00:00+00:00


In [104]:
df_reliefweb_situation_report.to_csv("c://temp//100_situation_reports.csv", encoding='utf-8-sig', index=False)

In [84]:
x = uuid.uuid4().hex
x

'7a823776db334c81960785e42707cccd'